# Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn 

In [4]:
from yelpapi import YelpAPI
import os, json, math, time
from tqdm.notebook import tqdm_notebook

# Credentials and Accessing the API



In [5]:
# Load API Credentials
with open ('/Users/ERNESTO/.secret/yelp_api.json') as f:
    login = json.load(f)
    
# Instantiate YelpAPI variable

yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)

# Define Search

To allow us to easily perform different searches in the future, we will define variables for LOCATION and TERM set for our particular search conditions. Then, when we want to use a different location or term, we can just redefine these variables. This streamlines the code and makes it more readable and reproducible.

In [6]:
LOCATION = 'NY, NY'
TERM = 'Pizza'

# Create a results-in-progress JSON file, but only if it doesn't exist.

This is the file where your results will be saved. Note: you must rename your JSON_FILE for different queries to prevent confusing results from other searches. We recommend you include your search terms in the filename.

In [7]:
# Specifying JSON_FILE filename (can include a folder)
# include the search term in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_NY_pizza.json'

# Check if our JSON_FILE already exists. 

This will prevent us from accidentally overwriting an existing file.

If it doesn't exist:

Create any folders needed for the file path.

Save an empty list as JSON_File

In [8]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


# Determine how many results are already in the file

Load the results file to determine the # of results we have previously retrieved. If you just created the file, you would expect it to be empty.

We will use this as our offset parameter for our API call. Even if this is your first API call, and the number is 0, we want to define "n_results" based on the length of "previous_results."

In [9]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


# Figure out how many pages of results we will need

We will perform our first query to get our first page of results and the total number of results. We will then (via code) calculate how many pages we will need to retrieve all of our results.

In [10]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many results total?
total_results = results['total']
total_results



12400

In [14]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

There are over 12400 businesses to retrieve from our API and we can get 20 results at a time (per "page").

We can calculate the # of results remaining by subtracting our offset (length of our previous results) from our total.

Then we can determining how many pages we will need by dividing the results by 20 (or whatever the value happens to be for results per page)

Note that we need to round up the number of pages in order to get all of the results. Even if there is only 1 result on the last page, we want to include that page! To do this we will use math.ceil.

In [15]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



620

When this example was written, there were 12400 results and 20 results per page. 12400 /20 = 620. But what if we had 12410 results? Rounding UP would give us 561 pages. We expect the first 560 pages to have 20 results and the last page to have the final 10 results. Notice that we have assigned the number of pages as n_pages here. We will use this value in our next segment of code.

You can see that having to manually go through 620 pages would be quite time consuming and inefficient! First we are going to save the first page into our file, and then we will add on to it with a for loop.

# Add this page of results to .json file

Our API returns our results in JSON format, with the businesses in a list of dictionaries. We will append the first page of businesses to our previous_results (which is very likely empty) and then save to disk.